In [1]:
using OMJulia;
using DataFrames;
omc= OMJulia.OMCSession();

import Pkg; Pkg.add("CSV"); using CSV

import Pkg; Pkg.add("Plots")
using Plots

plotly()

┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.0/OMJulia/TnZao.ji for OMJulia [b9c49fa3-9548-50ca-88f4-bcd0bd4a191f]
└ @ Base loading.jl:1190
┌ Warning: Package OMJulia does not have ZMQ in its dependencies:
│ - If you have OMJulia checked out for development and have
│   added ZMQ as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with OMJulia
└ Loading ZMQ into OMJulia from project dependency, future warnings for OMJulia are suppressed.


  Updating registry at `/opt/julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `/opt/julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.0/Manifest.toml`
 [no changes]


┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1192


 Resolving package versions...
 Installed RecipesBase ─ v0.6.0
 Installed PlotThemes ── v0.3.0
 Installed Plots ─────── v0.22.1
 Installed PlotUtils ─── v0.5.5
 Installed GR ────────── v0.36.0
  Updating `/opt/julia/environments/v1.0/Project.toml`
  [91a5bcdd] + Plots v0.22.1
  Updating `/opt/julia/environments/v1.0/Manifest.toml`
  [28b8d3ca] + GR v0.36.0
  [ccf2f8ad] + PlotThemes v0.3.0
  [995b91a9] + PlotUtils v0.5.5
  [91a5bcdd] + Plots v0.22.1
  [3cdcf5f2] + RecipesBase v0.6.0
  Building GR ───→ `/opt/julia/packages/GR/wFd9A/deps/build.log`
  Building Plots → `/opt/julia/packages/Plots/cDXot/deps/build.log`


┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1192


Plots.PlotlyBackend()

In [2]:
omc.sendExpression("loadModel(Modelica)")

"true\n"

In [101]:
omc.sendExpression("
    model a
       Real s,t,u,b; 
       parameter Real ppi = acos(-1);
    equation 
       s=sin(10*ppi*time); 
       b=rand()
       t=cos(250.0*ppi*time)/10;
       u=s+t;
    end a;
")

omc.sendExpression("simulate(a,outputFormat=\"csv\",stopTime=0.5,numberOfIntervals = 5000)")

"record SimulationResult\n    resultFile = \"/user/jovyan/work/a_res.csv\",\n    simulationOptions = \"startTime = 0.0, stopTime = 0.5, numberOfIntervals = 5000, tolerance = 1e-06, method = 'dassl', fileNamePrefix = 'a', options = '', outputFormat = 'csv', variableFilter = '.*', cflags = '', simflags = ''\",\n    messages = \"LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.\nLOG_SUCCESS       | info    | The simulation finished successfully.\n\",\n    timeFrontend = 0.2008101,\n    timeBackend = 0.0017425,\n    timeSimCode = 0.008791400000000001,\n    timeTemplates = 0.0303705,\n    timeCompile = 0.9440245,\n    timeSimulation = 0.1964153,\n    timeTotal = 1.3834569\nend SimulationResult;\n"

In [115]:
data = CSV.read("a_res.csv");

plot(data[:time],data[:s]+data[:t].*x)

In [92]:
import Random

In [111]:
x=Random.rand!(ones(size(data,1)));

In [105]:
names(data)

4-element Array{Symbol,1}:
 :time
 :s   
 :t   
 :u   

In [109]:
ones(size(data,1))

5002-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [117]:
omc.sendExpression("
model cylinder \"
+ When using the units packages (eg SIunits), the types specify immutable units.
  You have to use the Conversions subpackages to have values expressed in thems of other units.
+ displayUnit may be ignored by the modeling tool. OMPython in Jupyter Notebook 
  seems to ignore displayUnit.  
\"

  import Modelica.Constants.*;
//   import Modelica.SIunits.{Angle,AngularVelocity,Length,Diameter,Volume,Pressure,Temperature};
  import Modelica.SIunits.*;
//   import Modelica.SIunits.Conversions.NonSIunits.{Angle_deg,AngularVelocity_rpm};
  import Modelica.SIunits.Conversions.NonSIunits.*;
//   import Modelica.SIunits.Conversions.{from_deg,from_rpm}; 
  import Modelica.SIunits.Conversions.*; 
  import Modelica.Media.*;

//   type Volume = Real(final quantity = \"Volume\", final unit = \"l\");

     
  parameter AngularVelocity_rpm N_rpm=600 \"engine speed [rpm]\";
  parameter Diameter bore(displayUnit=\"mm\")=0.080 \"bore [m]\";
  parameter Length stroke(displayUnit=\"mm\")=0.090 \"stroke [m]\";
  parameter Length conrod(displayUnit=\"mm\")=0.140 \"connecting rod length [m]\";
  parameter Pressure P_bdc(displayUnit=\"bar\")=100000 \"pressure at BDC [Pa]\";
  parameter Temperature T_bdc=300 \"temp at BDC [K]\";
  parameter Real n=1.3 \"polytropic exponent\";
  parameter Real rc=12.5 \"compression ratio\";
  parameter Volume Vd(displayUnit=\"l\") = pi/4*bore^2*stroke \"displacement\";  
  parameter Volume V_tdc(displayUnit=\"l\") = Vd/(rc-1) \"clearance (min) volume\";
  parameter Volume V_bdc(displayUnit=\"l\") = V_tdc*rc \"max volume\";  
  parameter SpecificHeatCapacity R(displayUnit=\"J/(kg.K)\") = 8314/44 \"mass gas constant\";
  parameter Mass m_cyl(displayUnit=\"kg\") = P_bdc * V_bdc / (R * T_bdc) \"cylinder charge mass\";
//   parameter Density rho \"cylinder charge density\";
    
  package CO2 = Modelica.Media.IdealGases.SingleGases.CO2; 


// variables
  Angle_deg degCA(start=0) \"crank angle [deg]. Zero degrees is TDC\";
  Angle_deg degCAmod360 \"crank angle [deg]\";
  Angle CA \"crank angle [rad]\";
  Angle theta \"conrod angle from cylinder axis [rad]\";
  AngularVelocity N \"engine speed [rad/s]\";
  Volume V_cyl(displayUnit = \"l\") \"instantaneous cylinder volume\";
  Pressure P_cyl(displayUnit=\"bar\") \"cylinder pressure [Pa]\";
  Temperature T_cyl \"cylinder temperature [K]\";
  Temperature T_ig \"cylinder temperature [K]\";
  Length x(displayUnit=\"mm\") \"piston relative distance from TDC\";
  CO2.BaseProperties medium;  
  Density rho_cyl \"cylinder charge density\";
  Volume_litre V_cyl_litre \"instantaneous cylinder volume\";

  
// initial equation
//  degCA = 90;
   
equation
// define crank angle; display it in degrees, calculate it always in radians 
  CA = from_deg(degCA);
  N = from_rpm(N_rpm);
  der(CA) = N;
  degCAmod360 = mod(degCA,360);
  //degCA=mod(degCA,720);
  
// slider-crank geometry
  sin(CA)/conrod = sin(theta)/(stroke/2) \"Law of sines\";
  x = (( conrod+stroke/2 ))
     -(( conrod*cos(theta)+cos(CA)*stroke/2 )) \"x=0 @TDC; x=S @BDC\";
  V_cyl = V_tdc + x*pi/4*bore^2;
  V_cyl_litre = to_litre(V_cyl);
 
// let's add some thermodynamics
  P_cyl/P_bdc = (V_bdc/V_cyl)^n  \"polytropic relationship\";
  P_cyl*V_cyl/T_cyl = P_bdc*V_bdc/T_bdc \"ideal gas law\";
  rho_cyl = m_cyl/V_cyl;
//   state = CO2.setState_dT(rho_cyl, T_cyl);
  medium.p = P_cyl;
  medium.d = rho_cyl;
  T_ig = medium.T;
  
  
end cylinder;

")

"{cylinder}\n"

In [118]:
omc.sendExpression("simulate(cylinder,outputFormat=\"csv\",stopTime=0.5,numberOfIntervals = 5000)")

"record SimulationResult\n    resultFile = \"/user/jovyan/work/cylinder_res.csv\",\n    simulationOptions = \"startTime = 0.0, stopTime = 0.5, numberOfIntervals = 5000, tolerance = 1e-06, method = 'dassl', fileNamePrefix = 'cylinder', options = '', outputFormat = 'csv', variableFilter = '.*', cflags = '', simflags = ''\",\n    messages = \"LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.\nLOG_SUCCESS       | info    | The simulation finished successfully.\n\",\n    timeFrontend = 1.1516579,\n    timeBackend = 0.016503,\n    timeSimCode = 0.0233584,\n    timeTemplates = 0.0752507,\n    timeCompile = 1.2791205,\n    timeSimulation = 0.4409422,\n    timeTotal = 2.9882893\nend SimulationResult;\n"

In [119]:
cdata = CSV.read("cylinder_res.csv")


,time,CA,der(CA),N,P_cyl,T_cyl,T_ig,V_cyl,V_cyl_litre,degCA,degCAmod360,medium.MM,medium.R,medium.T_degC,medium.X[1],medium.h,medium.p_bar,medium.u,rho_cyl,theta,x,medium.T,medium.d,medium.p,medium.state.T,medium.state.p
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,0.0,0.0,62.8319,62.8319,2.66675e6,640.021,640.123,3.93382e-5,0.0393382,0.0,0.0,0.0440095,188.924,366.973,1,549739.0,26.6675,4.28804e5,22.0512,0.0,0.0,640.123,22.0512,2.66675e6,640.123,2.66675e6
2,0.0001,0.00628319,62.8319,62.8319,2.66623e6,639.992,640.094,3.93441e-5,0.0393441,0.36,0.36,0.0440095,188.924,366.944,1,5.49707e5,26.6623,4.28778e5,22.0479,0.00201958,1.17377e-6,640.094,22.0479,2.66623e6,640.094,2.66623e6
3,0.0002,0.0125664,62.8319,62.8319,2.66468e6,639.906,640.008,3.93618e-5,0.0393618,0.72,0.72,0.0440095,188.924,366.858,1,5.49613e5,26.6468,4.287e5,22.0379,0.0040391,4.69501e-6,640.008,22.0379,2.66468e6,640.008,2.66468e6
4,0.0003,0.0188496,62.8319,62.8319,2.66208e6,639.762,639.864,3.93913e-5,0.0393913,1.08,1.08,0.0440095,188.924,366.714,1,5.49455e5,26.6208,428569.0,22.0214,0.00605846,1.05635e-5,639.864,22.0214,2.66208e6,639.864,2.66208e6
5,0.0004,0.0251327,62.8319,62.8319,2.65846e6,639.561,639.663,3.94326e-5,0.0394326,1.44,1.44,0.0440095,188.924,366.513,1,5.49234e5,26.5846,4.28386e5,21.9984,0.00807762,1.87788e-5,639.663,21.9984,2.65846e6,639.663,2.65846e6
6,0.0005,0.0314159,62.8319,62.8319,2.65381e6,639.303,639.405,3.94857e-5,0.0394857,1.8,1.8,0.0440095,188.924,366.255,1,5.48951e5,26.5381,4.28152e5,21.9688,0.0100965,2.93405e-5,639.405,21.9688,2.65381e6,639.405,2.65381e6
7,0.0006,0.0376991,62.8319,62.8319,2.64816e6,638.988,639.09,3.95506e-5,0.0395506,2.16,2.16,0.0440095,188.924,365.94,1,5.48606e5,26.4816,4.27866e5,21.9328,0.012115,4.22477e-5,639.09,21.9328,2.64816e6,639.09,2.64816e6
8,0.0007,0.0439823,62.8319,62.8319,2.6415e6,638.617,638.719,3.96272e-5,0.0396272,2.52,2.52,0.0440095,188.924,365.569,1,5.48199e5,26.415,4.2753e5,21.8903,0.0141331,5.74998e-5,638.719,21.8903,2.6415e6,638.719,2.6415e6
9,0.0008,0.0502655,62.8319,62.8319,2.63385e6,638.19,638.292,3.97157e-5,0.0397157,2.88,2.88,0.0440095,188.924,365.142,1,5.47731e5,26.3385,4.27142e5,21.8416,0.0161507,7.50956e-5,638.292,21.8416,2.63385e6,638.292,2.63385e6


In [122]:
plot(cdata[:time],cdata[:T_ig])